# R2R RAG Quickstart (Linux)

This notebook shows how to use R2R for Retrieval-Augmented Generation (RAG):
- Install the Python SDK
- Verify the API is reachable
- Ingest a sample document
- Run search and RAG
- Try the agent for deeper analysis

To run, start the R2R API separately in a Linux shell.

## Prerequisites (run in a separate terminal)

1) Create a Python 3.12 venv and install the server (from this repo):
```bash
python3.12 -m venv .venv
source .venv/bin/activate
pip install -e ./py
```

2) Configure your providers (OpenAI-compatible):
```bash
# Text generation (e.g., LM Studio / OpenAI-compatible)
export LMSTUDIO_API_BASE="http://<gen-host>/api"   # or /v1 depending on server
export LMSTUDIO_API_KEY="<gen-token>"

# Embeddings (can be a different OpenAI-compatible base)
export OPENAI_API_BASE="http://<embed-host>/v1"
export OPENAI_API_KEY="<embed-token>"

# Postgres (adjust as needed)
export R2R_POSTGRES_HOST=localhost
export R2R_POSTGRES_USER=r2r
export R2R_POSTGRES_PASSWORD=r2rpassword
export R2R_POSTGRES_DBNAME=r2r
export R2R_PROJECT_NAME=r2r_default
```

Note: Ensure embedding dimensions in config match (see py/r2r/r2r.toml).

3) Start the API server:
```bash
export R2R_PORT=8002
python -m r2r.serve
# Server listens on http://0.0.0.0:8002
```

Then continue here in the notebook.

In [ ]:
# Install the Python SDK for the client
%pip -q install r2r requests

In [ ]:
# Point to your running R2R server
import os
BASE_URL = os.getenv('R2R_BASE_URL', 'http://localhost:8002')
print('Using BASE_URL =', BASE_URL)

In [ ]:
# Sanity check: hit the OpenAPI spec
import requests, json
spec_url = f{BASE_URL!r} + '/openapi_spec'
try:
    r = requests.get(spec_url, timeout=10)
    r.raise_for_status()
    print('OpenAPI available at', spec_url)
    # print a small part of the spec
    data = r.json() if r.headers.get('content-type','').startswith('application/json') else r.text
    if isinstance(data, dict):
        print('Title:', data.get('info', {}).get('title'))
    else:
        print(str(data)[:200] + '...')
except Exception as e:
    print('Failed to reach server:', e)
    print('Make sure the server is running as described above.')

In [ ]:
# Initialize the client
from r2r import R2RClient
client = R2RClient(base_url=BASE_URL)
client

## Ingest a sample document

You can upload your own `.pdf`, `.txt`, `.md`, etc.
Below we create a small `.txt` file and ingest it.

In [ ]:
# Create a small sample file
sample_path = 'sample_doc.txt'
with open(sample_path, 'w', encoding='utf-8') as f:
    f.write('DeepSeek R1 is a reasoning model. This file is a simple demo.\n')
    f.write('RAG combines retrieval with generation to produce grounded answers.')
print('Wrote', sample_path)

# Ingest the file
doc = client.documents.create(file_path=sample_path)
doc

In [ ]:
# List your documents
docs = client.documents.list()
len(docs), (docs[0] if docs else None)

## Search

Run a semantic or hybrid search over ingested content.

In [ ]:
search_res = client.retrieval.search(query='What is RAG?')
search_res

## RAG (with citations)

Ask a question and let R2R retrieve + generate an answer grounded in your documents.

In [ ]:
rag_res = client.retrieval.rag(query='Explain RAG briefly.')
rag_res

## Agentic RAG

Use the conversational agent with retrieval tools for richer, multi-step answers.

In [ ]:
agent_res = client.retrieval.agent(
    message={"role": "user", "content": "Give a short analysis of RAG."},
    rag_tools=["search_file_knowledge", "get_file_content"],
)
agent_res

## Advanced: custom search settings

You can pass `search_mode` and `search_settings` to control hybrid search, filters, etc.

In [ ]:
advanced = client.retrieval.rag(
    query='What does the sample say about RAG?',
    search_mode='advanced',
    search_settings={
        "use_hybrid_search": True,
        "hybrid_settings": {
            "full_text_weight": 1.0,
            "semantic_weight": 5.0,
            "full_text_limit": 50,
            "rrf_k": 50
        }
    }
)
advanced